<a href="https://colab.research.google.com/github/Cath-Strategic-Tech/adpdx_etl/blob/main/ADPDX_ClergyDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Enviro

In [1]:
# @title Global Variables { run: "auto", vertical-output: true, display-mode: "both" }

target_enviro = "adpdx_devpro" # @param {type:"string"}

diocesan_account_id_devpro = "001Dx00001CwMTQIA3" # @param {type:"string"}

## Import Libraries

In [2]:
# enviro setup

import pandas as pd

from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import userdata

from datetime import datetime
now = datetime.now()

try:
    from simple_salesforce import Salesforce
except:
    !pip install simple-salesforce
    from simple_salesforce import Salesforce

from IPython.display import clear_output

clear_output()

In [3]:
# ADPDX prod credentials
prod_pass = userdata.get('portland_prod_user')
prod_token = userdata.get('portland_prod_securitytiken')

# ADPDX dev_pro credentials
adpdx_devpro_pass = userdata.get('adpdx_devpro_pass')
adpdx_devpro_token = userdata.get('adpdx_devpro_token')

# instantiate a SF session object
sf = Salesforce(domain='test', username='matt+adpdx@meribahflow.com.devpro', password=adpdx_devpro_pass, security_token=adpdx_devpro_token)

### Load Salesforce xref data

In [4]:
# get all ACTIVE SF users

sf_users = sf.query('Select Alias, FirstName, LastName, Username, id from User WHERE IsActive = True')
df_sf_users = pd.DataFrame(sf_users['records'])
df_sf_users = df_sf_users.drop(columns = 'attributes')
df_sf_users

,Alias,FirstName,LastName,Username,Id
0,slum,Soames,Lum,soames+adpdx_devpro@meribahflow.com,005Dx000004DiZmIAK
1,DMark,Douglas,Markwell,dmarkwell@archdpdx.org.devpro,005Hu00000PmOgtIAF
2,autocln,None,Data.com Clean,automatedclean@00ddx0000002de7mam.ext,005Hu00000PmOh3IAF
3,mcirc,Marlin,Circo,mcirco@archdpdx.org.devpro,005Hu00000PmjBwIAJ
4,autoproc,Automated,Process,autoproc@00ddx0000002de7mam,005Hu00000QS489IAD
5,mmartin,Matthew,Martin,matt+adpdx@meribahflow.com.devpro,005Hu00000PmOh8IAF
6,Chatter,None,Chatter Expert,chatty.00ddx0000002de7mam.0pix2gvaezis@chatter...,005Hu00000QS48LIAT
7,platint,None,Platform Integration User,cloud@00ddx0000002de7mam,005Hu00000QS48OIAT


In [5]:
# get all SF Record Types
get_all_recordTypes = 'Select Id, Name, DeveloperName, sObjecttype, namespaceprefix from RecordType'

# get list of records, add to dataframe
sf_recordTypes = sf.query(get_all_recordTypes)
df_sf_recordTypes = pd.DataFrame(sf_recordTypes['records'])
df_sf_recordTypes = df_sf_recordTypes.drop(columns = 'attributes')

# Create a dictionary mapping 'DeveloperName' to 'Id' for faster lookup
record_types_mapping = df_sf_recordTypes.set_index('DeveloperName')['Id'].to_dict()

# df_sf_recordTypes.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/salesforce/sf_prod_recordTypes.csv', index=False, header=True)
df_sf_recordTypes

,Id,Name,DeveloperName,SobjectType,NamespacePrefix
0,012Dx0000009TJxIAM,Church,Church,Account,mbfc
1,012Dx0000009TJyIAM,Deanery,Deanery,Account,mbfc
2,012Dx0000009TJzIAM,Group,Group,Account,mbfc
3,012Dx0000009TK0IAM,Religious,Religious,Account,mbfc
4,012Dx0000009TK1IAM,School,School,Account,mbfc
5,012Dx0000009TK2IAM,Consecrated,Consecrated,Contact,mbfc
6,012Dx0000009TK3IAM,Lay Person,Lay_Person,Contact,mbfc
7,012Dx0000009TK4IAM,Permanent Deacon,Permanent_Deacon,Contact,mbfc
8,012Dx0000009TK5IAM,Priest,Priest,Contact,mbfc
9,012Dx0000009TK6IAM,Religious,Religious,Contact,mbfc


In [6]:
# get SF Account
get_all_accounts = 'Select id, Name, RecordTypeId, Type, mbfc__Parish_Code__c, Job_Id__c, Archdpdx_Migration_Id__c from Account'

# get list of records, add to dataframe
sf_accounts = sf.query(get_all_accounts)
df_sf_accounts = pd.DataFrame(sf_accounts['records'])
df_sf_accounts = df_sf_accounts.drop(columns = 'attributes')

df_sf_accounts.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/salesforce/sf_prod_accounts.csv', index=False, header=True)

df_sf_accounts

,Id,Name,RecordTypeId,Type,mbfc__Parish_Code__c,Job_Id__c,Archdpdx_Migration_Id__c
0,001Dx00001DLNJcIAP,Circo (Marlin) Household,012Hu000001pkqDIAQ,Household,None,None,None
1,001Dx00001DLKIAIA5,Universidad Del Norte,012Dx0000009TK1IAM,None,None,None,None
2,001Dx00001DL4HIIA1,O'Connor (Gerard) Household,012Hu000001pkqDIAQ,Household,None,None,None
3,001Dx00001DLJxSIAX,O'Brien (Peter) Household,012Hu000001pkqDIAQ,Household,None,None,None
4,001Dx00001DLMlkIAH,Colome (Cmar) Household,012Hu000001pkqDIAQ,Household,None,None,None
...,...,...,...,...,...,...,...
300,001Dx00001CwdP3IAJ,Yamhill County Vicariate,012Dx0000009TJyIAM,None,None,8,Deanery_18
301,001Dx00001CwrGZIAZ,Smith [DEMO] (John) Household,012Hu000001pkqDIAQ,Household,None,None,None
302,001Dx00001Cws2nIAB,Religious Sisters,012Dx0000009TK0IAM,None,None,None,None
303,001Dx00001CwrKCIAZ,ST. DEMO PARISH,012Dx0000009TJxIAM,None,None,None,None


In [7]:
# get SF Contacts
get_all_contacts = 'Select id, Name, npe01__Type_of_Account__c, RecordTypeId, CreatedById from Contact'

# get list of records, add to dataframe
sf_contacts = sf.query(get_all_contacts)
df_sf_contacts = pd.DataFrame(sf_contacts['records'])
df_sf_contacts = df_sf_contacts.drop(columns = 'attributes')

# df_sf_contacts.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/sf_prod_accounts.csv', index=False, header=True)

df_sf_contacts

,Id,Name,npe01__Type_of_Account__c,RecordTypeId,CreatedById
0,003Dx00000hGzugIAC,Marlin Circo,Individual,012Dx0000009TK3IAM,005Dx000004DiZmIAK
1,003Dx00000hGpqIIAS,Gerard O'Connor,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK
2,003Dx00000hGxz0IAC,Peter O'Brien,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK
3,003Dx00000hGzeiIAC,Cmar F Colome,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK
4,003Dx00000hGzn6IAC,Sharon Malone,Individual,012Dx0000009TK3IAM,005Dx000004DiZmIAK
5,003Dx00000hGzt4IAC,Hans Mueller,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK
6,003Dx00000hGztdIAC,Châu Pham,Individual,012Dx0000009TK4IAM,005Dx000004DiZmIAK
7,003Dx00000hGyCwIAK,Lucas Laborde,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK
8,003Dx00000hGztEIAS,John Henderson,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK
9,003Dx00000hGzx1IAC,David Kamau,Individual,012Dx0000009TK5IAM,005Dx000004DiZmIAK


## Load Functions

In [8]:
# Job ID Incrementer

def update_job_id(file_name):
    # Open the file in read mode and get the current job ID
    with open(file_name, 'r') as file:
        current_job_id = int(file.readline())

    # Increment the job ID
    new_job_id = current_job_id + 1

    # Open the file in write mode and update the job ID
    with open(file_name, 'w') as file:
        file.write(str(new_job_id))

    # Return the new job ID
    return new_job_id


# Concates two DF columns for an External ID

def concat_columns(df, columns, new_column, separator='_'):
    """
    Concatenates the values from specified columns into a single string
    with the specified separator and populates a new column in the DataFrame.

    Args:
    - df: pandas DataFrame
    - columns: list of column names to concatenate
    - new_column: name of the new column to be created
    - separator: separator to use between concatenated values (default is '_')

    Returns:
    - Updated pandas DataFrame with the new column
    """
    df[new_column] = df[columns].astype(str).apply(lambda x: separator.join(x), axis=1)
    return df



# ACCOUNTS

## Extract

### Load ArchdPDX csvs as DataFrames

In [9]:
df_offices = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Offices.csv', skiprows= lambda x: x in [1])
df_offices["src_table"] = 'Offices'
df_offices["AccountRecordType"] = 'Organization'
df_offices.rename({"Name": "Account Name"}, axis="columns", inplace=True)

df_parishes = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Parishes (3).csv', skiprows= lambda x: x in [1])
df_parishes["src_table"] = 'Parishes'
df_parishes["AccountRecordType"] = 'Church'
df_parishes.rename({"Parish Formal Name": "Account Name"}, axis="columns", inplace=True)

df_religious = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/RelCommunities.csv', skiprows= lambda x: x in [1])
df_religious["src_table"] = 'RelCommunities'
df_religious["AccountRecordType"] = 'Religious'
df_religious.rename({"Community Name": "Account Name"}, axis="columns", inplace=True)

df_schools = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Schools.csv', skiprows= lambda x: x in [1])
df_schools["src_table"] = 'Schools'
df_schools["AccountRecordType"] = 'School'
df_schools.rename({"School Name": "Account Name"}, axis="columns", inplace=True)

df_vicariates = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/Vicariates.csv', skiprows= lambda x: x in [1])
df_vicariates["src_table"] = 'Vicariates'
df_vicariates["AccountRecordType"] = 'Deanery'
df_vicariates.rename({"Vicariate Name": "Account Name"}, axis="columns", inplace=True)

df_newman = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/NewmanCenters.csv', skiprows= lambda x: x in [1])
df_newman["src_table"] = 'NewmanCenters'
df_newman["AccountRecordType"] = 'Organization'
df_newman.rename({"Newman Center Name": "Account Name", "Newman Center City": "Mailing Address City2"}, axis="columns", inplace=True)


### Merge DFs into a single Accounts DF

In [10]:
# init list of DataFrames
src_accounts = [df_offices, df_parishes, df_religious, df_schools, df_vicariates, df_newman]

# concats the various Account dataframes into one large table
accounts = pd.concat(src_accounts, ignore_index=True)

# renames columns headers to consolidate account names into SF-conformed data model
accounts.rename({"Common Name": "Name, City"}, axis="columns", inplace=True)

# reorder column order
col = accounts.pop('Account Name')
accounts.insert(2, col.name, col)

col = accounts.pop('Parish Name')
accounts.insert(3, col.name, col)

col = accounts.pop('AccountRecordType')
accounts.insert(1, col.name, col)

# set merged table as 'accounts'
src_accounts_merged = accounts

# export merged tables to CSV for viewing
src_accounts_merged.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/src_accounts_merged.csv')

# export merged tables DESCRIPTION to CSV for mapping
src_accounts_merged.describe(include='all').transpose().to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/src_accounts_merged_describe.csv')
src_accounts_merged.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Record Number,334.0,NaN,NaN,NaN,54.5,41.389801,1.0,21.25,45.0,76.75,173.0
AccountRecordType,334,5,Church,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Name, City",334,334,Pastoral Center,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Account Name,334,291,St. Mary,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parish Name,5,5,St. Anne,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Archdiocesan School Code,52,52,12-PDXARCS,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Grades Provided,52,12,PS-8,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address 1,56,55,4420 SW St Marys Dr,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mailing Address Zip,56.0,NaN,NaN,NaN,97222.446429,124.9586,97005.0,97134.75,97217.5,97301.0,97526.0


## Transform

- Strip phone numbers
- Validate email addresses
- Concat BillingStreet 1 + BillingStreet 2 into single field
- Vlookup - RecordTypeIds
-

### Rename Account columns to SF apis


In [11]:
#TODO

# print(accounts.columns)

accounts.rename(
    columns={
        'Account Name': 'Name',
        'Mailing Address': 'BillingStreet',
        'Mailing Address City': 'BillingCity',
        'Mailing Address State': 'BillingState',
        'Mailing Address Postal Code': 'BillingPostalCode',
        'Mailing Address Country': 'BillingCountry',
        'Email': 'mbfc__Email__c',
        'Web Site': 'Website',
        'Order Common Name': 'mbfc__Abbreviation__c',
        'Order Letters': 'mbfc__Religious_Suffix__c',
        'Men or Women': 'mbfc__Type_Members__c'
    },
    inplace=True
)

print(accounts.columns)

Index(['Record Number', 'AccountRecordType', 'Name, City', 'Name',
       'Parish Name', 'Archdiocese Assigns Clergy', 'Locator Description',
       'BillingStreet', 'Mailing Address 2', 'BillingCity', 'BillingState',
       'Mailing Address Province', 'BillingPostalCode', 'BillingCountry',
       'Phone', 'Fax', 'mbfc__Email__c', 'Website', 'src_table', 'Sort Name',
       'Parish City', 'Mission Of', 'Established', 'Vicariate', 'Non-Latin',
       'County', 'Disabled Access', 'Sanctuary Capacity',
       'Lat/Long Coordinates Decimal', 'Google Small Embed URL',
       'Miles to Pastoral Center', 'Schedule 1 Head', 'Schedule 1 Text',
       'Schedule 2 Head', 'Schedule 2 Text', 'Schedule 3 Head',
       'Schedule 3 Text', 'Schedule 4 Head', 'Schedule 4 Text',
       'Schedule 5 Head', 'Schedule 5 Text', 'Schedule 6 Head',
       'Schedule 6 Text', 'Schedule 7 Head', 'Schedule 7 Text',
       'Community City', 'Order Full Name', 'mbfc__Abbreviation__c',
       'mbfc__Religious_Suffix__

In [12]:
# Create a single BillingAddress field

# billingstreet = str(f"Mailing Addresss 1 /n Mailing Address 2")

# accounts['BillingStreet'] = accounts['Mailing Address 1'].astype(str) + accounts['Mailing Address 2'].astype(str)

accounts

,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,Mailing Address 2,BillingCity,...,Major Superior Phone,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Mailing Address City2
0,1,Organization,Pastoral Center,Pastoral Center,NaN,Yes,NaN,2838 E Burnside St,NaN,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Organization,Catholic Sentinel,Catholic Sentinel,NaN,No,NaN,2838 E Burnside St,NaN,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Organization,Catholic Cemeteries,Catholic Cemeteries,NaN,No,NaN,333 SW Skyline Blvd,NaN,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,Organization,Griffin Center,Griffin Center,NaN,No,NaN,11957 SE Fuller Rd,NaN,Milwaukie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Organization,Providence Portland Medical Center,Providence Portland Medical Center,NaN,Yes,NaN,4805 NE Glisan St,NaN,Portland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,18,Deanery,Yamhill County Vicariate,Yamhill County,NaN,Yes,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,1,Organization,"OSU Newman Center, Corvallis",OSU Newman Center,NaN,No,NaN,2127 NW Monroe Ave,NaN,Corvallis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Corvallis
331,2,Organization,"St. Thomas More (UO) Newman Center, Eugene",St. Thomas More (UO) Newman Center,NaN,No,NaN,1850 Emerald St,NaN,Eugene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eugene
332,3,Organization,Walsh Memorial (SOU) Newman Center at Our Lady...,Walsh Memorial (SOU) Newman Center at Our Lady...,NaN,Yes,NaN,987 Hillview Dr,NaN,Ashland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ashland


### TODO: Set mbfc__Church_Type__c

In [13]:
accounts.loc[accounts['AccountRecordType'] == 'Church', 'mbfc__Church_Type__c'] = 'Parish'

accounts[accounts['AccountRecordType'] == 'Church']

# TODO: Handle churches that aren't parishes (missions, non-diocesan parishes, etc.)

,Record Number,AccountRecordType,"Name, City",Name,Parish Name,Archdiocese Assigns Clergy,Locator Description,BillingStreet,Mailing Address 2,BillingCity,...,Major Superior Email,School City,Parish Link,Vicariate Link,Archdiocesan School Code,Grades Provided,Mailing Address 1,Mailing Address Zip,Mailing Address City2,mbfc__Church_Type__c
35,1,Church,"Our Lady of Perpetual Help, St Mary’s, Albany","Our Lady of Perpetual Help, St Mary’s",NaN,Yes,SW Ellsworth St between 8th and 9th Streets,"Our Lady of Perpetual Help, St Mary’s Parish",815 Broadalbin St SW,Albany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
36,2,Church,"St. Andrew Dũng-Lạc Mission, Aloha",St. Andrew Dũng-Lạc,NaN,No,SW Grabhorn Rd/209th Ave and Farmington Rd,St. Andrew Dũng-Lạc Mission,7390 SW Grabhorn Rd,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
37,3,Church,"St. Elizabeth Ann Seton, Aloha",St. Elizabeth Ann Seton,NaN,Yes,NaN,St. Elizabeth Ann Seton Parish,3145 SW 192nd Ave,Aloha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
38,4,Church,"St. Peter the Fisherman Mission, Arch Cape",St. Peter the Fisherman,NaN,Yes,79441 Hwy 101 S,St. Peter the Fisherman Mission,PO Box 29,Seaside,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
39,5,Church,"Our Lady of the Mountain, Ashland",Our Lady of the Mountain,NaN,Yes,NaN,Our Lady of the Mountain Parish,987 Hillview Dr,Ashland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,152,Church,"St. John in the Woods Mission, Welches",St. John in the Woods,NaN,Yes,24905 E Woodsey Way,St. John in the Woods Mission,18090 SE Langensand Rd,Sandy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
182,153,Church,"St. Cyril, Wilsonville",St. Cyril,NaN,Yes,9210 SW 5th St,St. Cyril Parish,9205 SW 5th St,Wilsonville,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
183,154,Church,"St. Luke, Woodburn",St. Luke,NaN,Yes,NaN,St. Luke Parish,417 Harrison St,Woodburn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish
184,155,Church,"St. John the Evangelist, Yamhill",St. John the Evangelist,NaN,Yes,NaN,St. John the Evangelist Parish,445 N Maple St,Yamhill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parish


### Generate an External ID

In [14]:
# set columns to concatenate
columns_to_concate = ['AccountRecordType', 'Record Number']

# call UDF
accounts = concat_columns(accounts, columns_to_concate, 'Archdpdx_Migration_Id__c', separator='_')

### Merge in RecordTypeIds

In [15]:
accounts['RecordTypeId'] = accounts['AccountRecordType'].map(record_types_mapping)

### TODO: For all Accounts with 'Primary Contacts' or 'Major Superiors', add Contact to staging DF, to be created after parent account is created.

### Export working file to CSV

In [16]:
accounts.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/accounts_working.csv')


## Load


### Get a new Job ID



In [17]:
# increment to the job_id
file_name = '/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/jobs/job_id'
new_job_id = update_job_id(file_name)
print(f"New job ID: {new_job_id}")

# add/update account DF with job_id
accounts["Job_Id__c"] = new_job_id


New job ID: 19


### A) Vicariates

In [18]:
vicariates = accounts[accounts['AccountRecordType'] == 'Deanery']

vicariates = vicariates[[
    'Record Number',
    'Name, City',
    'AccountRecordType',
    'Archdiocese Assigns Clergy',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c',
    'RecordTypeId'
    ]]

# add parentid
vicariates["mbfc__Diocese__c"] = diocesan_account_id_devpro
vicariates['ParentId'] = diocesan_account_id_devpro

vicariates.rename(columns={
        'Name, City': 'Name',
        'External_Id': 'Archdpdx_Migration_Id__c'
    }, inplace=True)

# export to CSV
vicariates.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/vicariates_staging.csv')

# create a map of Vicariate lookup ids to the unique ids generated on the Vicariate records
vicariates_externalid_map = vicariates.set_index('Record Number')['Archdpdx_Migration_Id__c'].to_dict()

vicariates_externalid_map

{1: 'Deanery_1',
 2: 'Deanery_2',
 3: 'Deanery_3',
 4: 'Deanery_4',
 5: 'Deanery_5',
 6: 'Deanery_6',
 7: 'Deanery_7',
 8: 'Deanery_8',
 9: 'Deanery_9',
 10: 'Deanery_10',
 11: 'Deanery_11',
 12: 'Deanery_12',
 13: 'Deanery_13',
 14: 'Deanery_14',
 15: 'Deanery_15',
 16: 'Deanery_16',
 17: 'Deanery_17',
 18: 'Deanery_18'}

In [19]:
# @title Get Vicariate records from SF

sf_deaneries = sf.query("SELECT Archdpdx_Migration_Id__c, Id FROM Account WHERE RecordType.DeveloperName = 'Deanery'")

df_sf_deaneries = pd.DataFrame(sf_deaneries['records'])
df_sf_deaneries = df_sf_deaneries.drop(columns = 'attributes')

df_sf_deaneries

# Creates a dict of Vicariate unique ids to the new Salesforce record IDs, so can populate on latter Account records
vicariate_sf_recordids = df_sf_deaneries.set_index('Archdpdx_Migration_Id__c')['Id'].to_dict()
vicariate_sf_recordids

{'Deanery_1': '001Dx00001CwdOmIAJ',
 'Deanery_2': '001Dx00001CwdOnIAJ',
 'Deanery_3': '001Dx00001CwdOoIAJ',
 'Deanery_4': '001Dx00001CwdOpIAJ',
 'Deanery_5': '001Dx00001CwdOqIAJ',
 'Deanery_6': '001Dx00001CwdOrIAJ',
 'Deanery_7': '001Dx00001CwdOsIAJ',
 'Deanery_8': '001Dx00001CwdOtIAJ',
 'Deanery_9': '001Dx00001CwdOuIAJ',
 'Deanery_10': '001Dx00001CwdOvIAJ',
 'Deanery_11': '001Dx00001CwdOwIAJ',
 'Deanery_12': '001Dx00001CwdOxIAJ',
 'Deanery_13': '001Dx00001CwdOyIAJ',
 'Deanery_14': '001Dx00001CwdOzIAJ',
 'Deanery_15': '001Dx00001CwdP0IAJ',
 'Deanery_16': '001Dx00001CwdP1IAJ',
 'Deanery_17': '001Dx00001CwdP2IAJ',
 'Deanery_18': '001Dx00001CwdP3IAJ'}

### B) Parishes, Schools, Organizations

In [20]:
acc_main = accounts[accounts['AccountRecordType'] != 'Deanery']
acc_main = acc_main[acc_main['AccountRecordType'] != 'Religious']

acc_main[acc_main['AccountRecordType'] == 'Church']['Vicariate'].astype(int)

# Pull in Vicariate record ids that match unique ids
acc_main['VicariateUniqueId'] = acc_main['Vicariate'].map(vicariates_externalid_map)
acc_main['mbfc__Deanery__c'] = acc_main['VicariateUniqueId'].map(vicariate_sf_recordids)

acc_main[acc_main['AccountRecordType'] == 'Church']['mbfc__Deanery__c']


35     001Dx00001CwdOmIAJ
36     001Dx00001CwdOyIAJ
37     001Dx00001CwdP1IAJ
38     001Dx00001CwdOuIAJ
39     001Dx00001CwdP0IAJ
              ...        
181    001Dx00001CwdOqIAJ
182    001Dx00001CwdP2IAJ
183    001Dx00001CwdOrIAJ
184    001Dx00001CwdP3IAJ
185    001Dx00001CwdOsIAJ
Name: mbfc__Deanery__c, Length: 151, dtype: object

In [21]:
# @title Export to CSV
# Export to CSV for manual loading

accounts_staging = acc_main[[
    'Name',
    'RecordTypeId',
    'mbfc__Church_Type__c',
    'mbfc__Deanery__c',
    'BillingStreet',
    'BillingCity',
    'BillingState',
    'BillingPostalCode',
    'BillingCountry',
    'Phone',
    'Fax',
    'mbfc__Email__c',
    'Website',
    'mbfc__Abbreviation__c',
    'mbfc__Religious_Suffix__c',
    'mbfc__Type_Members__c',
    'Description',
    'Job_Id__c',
    'Archdpdx_Migration_Id__c'

    ]]

accounts_staging.to_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/accounts_staging.csv')

In [22]:
# @title Extract SF Account records

sf_accounts = sf.query('Select id, Name, RecordTypeId, mbfc__Church_Type__c, Archdpdx_Migration_Id__c, Job_Id__c from Account WHERE Job_Id__c != null')
sf_accounts = pd.DataFrame(sf_accounts['records'])
sf_accounts = sf_accounts.drop(columns = 'attributes')
# sf_accounts

### C) TODO: Religious Institutes



In [23]:
df_religious_acc = acc_main[acc_main['AccountRecordType'] != 'Religious']

### D) TODO: Religious Communities

# Contacts

## Extract

In [24]:
df_contacts = pd.read_csv('/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/reports from clergypdx/People.csv')


## Analysis

Here we check the various columns and their types, count where values exist, count of unique values, sample data, etc.

DF shape:
- 142 columns
- 3017 rows

In [25]:
# Check the original shape of the imported CSV
print(f"Shape of original data set: {df_contacts.shape}")

# export to csv a list of the contact fields with count, unique, top, freq
contacts_describe = df_contacts.describe(include='all').transpose()
contacts_describe.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/analysis/contacts_describe.csv')

df_contacts.describe(include='all').transpose()  #initial analysis of the Contacts table

Shape of original data set: (3017, 142)


,count,unique,top,freq
Record Number,3017,3017,recNum,1
Common Name,3017,3012,Ms. Leslie Jones,2
Sort Name,3017,3010,nguyen anthony,3
Type(s),3017,30,Staff,1139
Clergy Status,1139,9,Transferred Out,462
...,...,...,...,...
Place of Work,270,134,Mount Angel Abbey,37
Volunteer Place,55,48,Mary’s Woods,4
Type of Work,277,118,Pastoral Ministry,30
Work Load,263,3,Full Time,230


In [35]:
df_contacts.Faculties.value_counts()

Faculties
General         234
Confessional    124
Diaconal         92
Restricted        7
facul             1
Name: count, dtype: int64

## Transform

### Rename Columns to SF apis

In [26]:
df_contacts.rename(
    columns={
    },
    inplace=True
)

### Set Contact Record Type

In [27]:
# Add new Contact Record Type column
df_contacts['ContactRecordType'] = ""
col = df_contacts.pop('ContactRecordType')
df_contacts.insert(4, col.name, col)
df_contacts

,Record Number,Common Name,Sort Name,Type(s),ContactRecordType,Clergy Status,Religious Status,Login ID,Password,Password Must be Changed,...,CARA Ethnicity,Seminarian Status,Other Diaconal Ministry,Spiritual Director Authorized,Link to Religious Community,Place of Work,Volunteer Place,Type of Work,Work Load,Work Title
0,recNum,comName,sortName,type,,clgyStatus,relStatus,logId,pwd,pwdChgYN,...,ethn,semStat,othDiacMin,spDirAuthYN,relPerRcmLink,relPlaceWork,relPlaceVol,relTypeWork,relWorkLoad,relTitleWork
1,2766,Rev. Stephen Abaukaka,abaukaka stephen ozovehe,Priest,,Transferred Out,NaN,sabukaka,def2a990be60a7998b1ed7c820101f3bd02d33b8992518...,Yes,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2337,Mr. Rogelio Acevedo,acevedo rogelio,Staff,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,3244,Mr. Sean Ackroyd,ackroyd sean,Staff,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,3295,Ms. Sherril Acton,acton sherril,Staff,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,1670,Ms. Jenny Zomerdyk,zomerdyk jenny,Staff,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3013,2755,"Br. Daniel Zorrilla, MSpS",zorrilla daniel,Religious,,NaN,Active,dzorrilla,391eedf7c936f63d3d0a7d9ea7e506a84709662fd31ba9...,Yes,...,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN
3014,1962,Ms. Kim Zuber,zuber kim,Staff,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3015,2202,Ms. Agnes Zueger,zueger agnes,Staff,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [28]:
# print(df_contacts.columns.tolist())

In [29]:
# Set Record Type

# Go down row by row and check the 'Type(s)' columns, check for certain words that are keys in a dictionary, and
# the that row's 'Type(s)' field contains a string that is in the a key in a dictionary the update another columns
# called 'ContactRecordType' with the paired value.

contact_type_map = {
    'Bishop': 'Priest',
    'Diaconate': 'Lay_Person',
    'Permanent Deacon': 'Permanent_Deacon',
    'Priest': 'Priest',
    'Staff': 'Lay_Person',
    'Seminarian': 'Lay_Person',
    'Wife': 'Lay_Person',
    'Religious': 'Religious'
}

def update_contact_record_type(row):
    for key, value in contact_type_map.items():
        if key in row['Type(s)']:
            row['ContactRecordType'] = value
            break
    return row

df_contacts = df_contacts.apply(update_contact_record_type, axis=1)

## Load

In [30]:
# generate CSV for manual loading
df_contacts.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/staging/contacts_staging.csv')

# Users

In [31]:
df_users = df_contacts[df_contacts['Access Permission'].isna() == False]
df_users = df_users[['Record Number', 'Common Name', 'Sort Name', 'Type(s)', 'ContactRecordType', 'Login ID', 'Access Permission']]
df_users.sort_values('Access Permission')
df_users.to_csv(f'/content/drive/Shareddrives/Clients/ADPDX (Portland)/Data/Clergy DB/working/users_working.csv')

# Assignments




## TODO
- Decide on unique external ID (eg. 'Assignments_' + recNum)
- new column = concat 'tableName' + 'tableLink' to get a unique ID for the Accounts staging table
- Update mapping document
    - if archAsgYN == Yes, then Affiliation.Type = 'Assignment'


## Questions
- For non-Archdiocesan assignments: How stale does this data appear to be? Group by parish, date create.

# Data Migration Best Practices (Notes)

### Understand the Source Data, Ask Questions to Understand Intent, Analyze Data to Know the Reality
1. Start with a conversation with business users. Maybe have them demo the current system (aka “the source system”) as it is used today. Take notes and ask questions related to the data model, but in business terms.
1. After you have a general understanding of what you are looking for, you can start your data analysis. Start by asking for (and acquiring) all the technical documentation on the current system (ERDs, data dictionaries, user guides, and so on) and access to the system’s front and back ends.
1. To understand the current system's data model, create a **Source Data Dictionary**: Create a spreadsheet with two tabs: one that lists all the objects/ tables and one that has all the fields. If you can get record counts by table, and/or population densities by field, include that data as well.

### Document Your Transformations

1. Start with your source data dictionary Objects/Tables tab, and copy the objects needed to a new spreadsheet. Add a column for the source table and target Salesforce objects, and another for notes. Then, fill these columns, including your notes that track all your open questions carefully.
1. Next, move on to field-level mapping. Start a new worksheet in your data mapping document for field-level mapping. From the source data dictionary Fields tab, copy all the field data for all the objects/tables needed for your migration. Then, to the left of that, add columns for transformation notes as well as Salesforce target information
1. **NOTE:** You must keep this document up to date with every single change that is made! If a defect is reported (during QA or UAT), update your data mapping document then fix your code. No joke. Do it immediately. It’s immensely important that your documentation remain the source of truth for all transformation rules.

